In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import re
import glob
import csv

import time

import nltk

from sklearn.externals import joblib

In [4]:
df = pd.read_csv('data/MeTooSTEM.csv')
print(df.head())

                    id      conversation_id     created_at        date  \
0  1115338964397895680  1115338964397895680  1554752505000  2019-04-08   
1  1115335506097836033  1115335506097836033  1554751681000  2019-04-08   
2  1115334978781569024  1115334978781569024  1554751555000  2019-04-08   
3  1115330819185369100  1115330819185369100  1554750563000  2019-04-08   
4  1115322598139867139  1115322598139867139  1554748603000  2019-04-08   

       time timezone             user_id         username             name  \
0  15:41:45      EDT  834812681672605696         ljuba_pi      Liuba Papeo   
1  15:28:01      EDT          3148749695  falla_alejandra  Alejandra Falla   
2  15:25:55      EDT           340277194   chelsey_rhodes   Chelsey Rhodes   
3  15:09:23      EDT           151161683  reneeisfabulous            Renee   
4  14:36:43      EDT  968873186786979840      marionhogg2      Marion Hogg   

  place        ...        retweet gif_url gif_thumb video_url video_thumb  \
0   NaN  

In [5]:
print(df.tail())

                      id     conversation_id     created_at        date  \
8896  955468224203755523  955468224203755523  1516636351000  2018-01-22   
8897  955097730765991937  955097730765991937  1516548018000  2018-01-21   
8898  951807927190466561  951798169276436480  1515763668000  2018-01-12   
8899  939256988101431296  939256988101431296  1512771291000  2017-12-08   
8900  939253524059344896  939253524059344896  1512770465000  2017-12-08   

          time timezone             user_id       username  \
8896  10:52:31      EDT          4562173852      sciencevs   
8897  10:20:18      EDT           298619241    drcswilliam   
8898  08:27:48      EDT  787836169811746820  shittymenlist   
8899  17:14:51      EDT           298619241    drcswilliam   
8900  17:01:05      EDT           298619241    drcswilliam   

                     name place        ...        retweet gif_url gif_thumb  \
8896           Science Vs   NaN        ...            NaN     NaN       NaN   
8897  Claudia Will

In [6]:
df.describe()

,id,conversation_id,created_at,user_id,replies_count,retweets_count,likes_count,location,retweet,is_quote_status,quote_id,is_reply_to,has_parent_tweet
count,8.901000e+03,8.901000e+03,8.901000e+03,8.901000e+03,8901.000000,8901.000000,8901.000000,0.0,0.0,8901.000000,8.901000e+03,8901.000000,8901.000000
mean,1.066191e+18,1.065554e+18,1.543035e+12,3.032811e+17,0.720930,4.280643,14.493315,NaN,NaN,0.316706,3.374774e+17,0.256376,0.256376
std,3.318776e+16,3.514883e+16,7.912578e+09,4.417201e+17,3.725931,19.115036,111.482696,NaN,NaN,0.465218,4.961886e+17,0.436656,0.436656
min,9.392535e+17,2.045999e+17,1.512770e+12,1.024200e+04,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000e+00,0.000000,0.000000
25%,1.045350e+18,1.044248e+18,1.538066e+12,5.047823e+08,0.000000,0.000000,1.000000,NaN,NaN,0.000000,0.000000e+00,0.000000,0.000000
50%,1.067856e+18,1.067644e+18,1.543432e+12,2.155745e+09,0.000000,1.000000,3.000000,NaN,NaN,0.000000,0.000000e+00,0.000000,0.000000
75%,1.096993e+18,1.096741e+18,1.550379e+12,8.512929e+17,1.000000,3.000000,10.000000,NaN,NaN,1.000000,1.036685e+18,1.000000,1.000000
max,1.115339e+18,1.115339e+18,1.554753e+12,1.113899e+18,252.000000,901.000000,8882.000000,NaN,NaN,1.000000,1.115305e+18,1.000000,1.000000


In [8]:
df.dtypes

id                    int64
conversation_id       int64
created_at            int64
date                 object
time                 object
timezone             object
user_id               int64
username             object
name                 object
place                object
tweet                object
tags                 object
replies              object
mentions             object
urls                 object
photos               object
replies_count         int64
retweets_count        int64
likes_count           int64
location            float64
hashtags             object
link                 object
retweet             float64
gif_url              object
gif_thumb            object
video_url            object
video_thumb          object
is_quote_status       int64
quote_id              int64
quote_url            object
is_reply_to           int64
has_parent_tweet      int64
dtype: object

In [10]:
print(df['tweet'].head)

<bound method NDFrame.head of 0       “The biggest success of the #MeTooSTEM site is...
1       Muchas gracias a @mariamonic91 y al @elespecta...
2       Awesome interview with @McLNeuro on #MeTooSTEM...
3       BethAnn McLaughlin: ‘Too many women in science...
4       Great talk from Celeste Kidd today highlightin...
5       So pumped to finally release that @MeTooSTEM i...
6       Watch out NYC! #MeTooSTEM is coming to town!  ...
7               #MeTooSTEM  https://youtu.be/rMqayQ-U74s 
8       I am so excited to march with @McLNeuro and @M...
9       Have written @robertsala63, dir of @iphes in T...
10      "What does redemption look like? Is there a wa...
11      Getting rid of @DataCamp until they rectify wh...
12      “Harassment of women in #STEM is the worst of ...
13      We're excited to participate in the #MeTooSTEM...
14      “The biggest success of the #MeTooSTEM site is...
15      Organizing some books for @McLNeuro's Seattle ...
16      at Apr 6 #ExpBio @APSPhysiology @S

In [13]:
df['text'] = df['tweet']

In [14]:
# 'http' matches literal characters
# '\S+' matches all non-whitespace characters (the end of the url)

# remove http links
df['text'] = df['text'].str.replace('http\S+', '', case=False)

# remove www links
df['text'] = df['text'].str.replace('www.\S+', '', case=False)

# remove twitter pics
df['text'] = df['text'].str.replace('pic.twitter.com\S+', '', case=False)

In [15]:
# remove hashtags
df['text'] = df['text'].str.replace('#\S+', '', case=False)

# remove users
df['text'] = df['text'].str.replace('@\S+', '', case=False)

# remove …
df['text'] = df['text'].str.replace('…','')

In [16]:
print(df['text'].head)

<bound method NDFrame.head of 0       “The biggest success of the  site is in us act...
1       Muchas gracias a  y al  por compartir mi histo...
2                   Awesome interview with  on  issues   
3       BethAnn McLaughlin: ‘Too many women in science...
4       Great talk from Celeste Kidd today highlightin...
5       So pumped to finally release that  is going to...
6                    Watch out NYC!  is coming to town!  
7                                                        
8       I am so excited to march with  and  at  NYC pe...
9       Have written  dir of  in Tarragona, asking why...
10      "What does redemption look like? Is there a wa...
11      Getting rid of  until they rectify what they h...
12      “Harassment of women in  is the worst of any s...
13      We're excited to participate in the  workshops...
14      “The biggest success of the  site is in us act...
15      Organizing some books for  Seattle workshop.  ...
16      at Apr 6    symposium, was happy 2

In [18]:
print(df.head())

                    id      conversation_id     created_at        date  \
0  1115338964397895680  1115338964397895680  1554752505000  2019-04-08   
1  1115335506097836033  1115335506097836033  1554751681000  2019-04-08   
2  1115334978781569024  1115334978781569024  1554751555000  2019-04-08   
3  1115330819185369100  1115330819185369100  1554750563000  2019-04-08   
4  1115322598139867139  1115322598139867139  1554748603000  2019-04-08   

       time timezone             user_id         username             name  \
0  15:41:45      EDT  834812681672605696         ljuba_pi      Liuba Papeo   
1  15:28:01      EDT          3148749695  falla_alejandra  Alejandra Falla   
2  15:25:55      EDT           340277194   chelsey_rhodes   Chelsey Rhodes   
3  15:09:23      EDT           151161683  reneeisfabulous            Renee   
4  14:36:43      EDT  968873186786979840      marionhogg2      Marion Hogg   

  place                        ...                         gif_url gif_thumb  \
0   Na

In [19]:
df['name'].value_counts()

McLNeuro                                  1064
MeTooSTEM                                  403
boyswillbeaccountable                      372
Kalynda Gonzales Stokes PhD                201
Michael Balter                             179
CharterForHealthcare                       147
Amelia McLNeuro                            127
Dr Mel Thomson                             100
Tisha Bohr 🧫🔬                               94
deanna دينا                                 79
Dr. Teresa Swanson                          78
Julie Libarkin                              69
Dr. Sherry Marts                            68
Swedish Doctors' Metoo Movement             63
Debra DeLoach 🍄🐿️                           58
Angie NMNH                                  58
Ana Safavi, MD                              58
Kelsey Priest, PhD, MPH                     57
Sister                                      53
ADVANCEGeo                                  52
Emily Myers                                 51
Gary McDowell

In [20]:
df['date'].value_counts()

2018-06-12    331
2018-10-14    138
2018-11-03    129
2019-02-27    120
2018-06-26    118
2019-02-28    115
2019-02-15    114
2019-02-22    110
2018-10-13     99
2018-12-13     98
2018-10-12     95
2019-02-26     94
2019-04-06     88
2018-11-04     87
2018-11-09     81
2019-01-31     80
2019-02-20     77
2018-10-16     75
2019-02-21     73
2019-02-14     73
2019-03-01     72
2018-11-15     72
2019-02-13     72
2018-11-06     71
2018-10-11     70
2019-02-16     66
2019-03-13     65
2018-11-27     64
2018-11-30     63
2018-11-16     63
             ... 
2018-12-15      6
2018-07-26      6
2018-07-10      6
2018-07-05      6
2018-08-16      6
2018-08-25      6
2018-12-24      5
2018-09-08      5
2018-07-07      5
2018-09-14      5
2018-08-30      5
2018-07-25      5
2018-09-02      4
2018-08-11      3
2018-09-09      3
2018-08-02      3
2018-07-31      3
2018-08-15      3
2018-08-01      2
2018-01-22      2
2017-12-08      2
2018-05-30      2
2018-10-27      2
2018-02-04      1
2018-05-28